### Imports

In [1]:
import numpy as np
import pandas as pd
from pyproj import Transformer
from datetime import date,datetime

In [224]:
#Import general library
import bokeh

#Import to handle data parsed to bokeh plots
from bokeh.models import ColumnDataSource

#Import to handle bokeh notebook
from bokeh.io import output_notebook

#Import for the background of the map plot
from bokeh.tile_providers import CARTODBPOSITRON, get_provider

#Import to create plot, show plot, and output plot
from bokeh.plotting import figure, output_file, show

#Import to handle layout of the bokeh plots
from bokeh.layouts import column, row, layout

#Importing the RangeTool used in the Timeline plot
from bokeh.models import RangeTool

#Import to set the range of the Timeline
from bokeh.models import Range1d

output_notebook()

Loading BokehJS ...

### Assumptions

In [88]:
#screen resolution
screen_height = 680
screen_length = 1208

### Data import and preparation

In [197]:
#Columns that we want to keep from the data we load in
import_columns = ['created_at_CET','latitude','longitude','Overall.score','Anger','Confusion',
                  'Disgust','Fear','Happiness','Sadness','Shame','Surprise']

#Load the data into the python environment
twitter_data = pd.read_csv("D:/Master Project/complete_swiss_dataset.csv",encoding = "ISO-8859-15",low_memory=False)

#Select only the columns that are relevant
#Timeline Map Histogram data (TMH_data)
TMH_data = twitter_data[import_columns].sort_values(by='created_at_CET',ignore_index=True)

#Round all entries in dataframe
#Mainly to remove specific gps locations of users
TMH_data = time_data.round(3)

#Format the string input to datetime
TMH_data['created_at_CET'] = pd.to_datetime(TMH_data['created_at_CET'], format='%Y-%m-%d %H:%M:%S')

#Sort the data by the date at creation
TMH_data = TMH_data.sort_values(by='created_at_CET')

#Removing the entire dataset now that the selection has been made
twitter_data = None

In [198]:
#Declare transformer to convert from epsg4326 to epsg 3856 (web mercator)
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857")

mercator_transform = transformer.transform(TMH_data['latitude'].values,
                                           TMH_data['longitude'].values)

TMH_data['x_mercator'] = mercator_transform[0].round(1)
TMH_data['y_mercator'] = mercator_transform[1].round(1)

In [199]:
%%time
TMH_data

Wall time: 0 ns


,created_at_CET,latitude,longitude,Overall.score,Anger,Confusion,Disgust,Fear,Happiness,Sadness,Shame,Surprise,x_mercator,y_mercator
0,2015-01-01 01:00:03,47.458,8.548,0,0,0,0,0,0,0,0,0,951559.0,6017154.4
1,2015-01-01 01:00:09,46.160,8.777,0,0,0,0,0,0,0,0,0,977051.2,5806026.5
2,2015-01-01 01:00:14,46.360,6.929,3,0,0,0,0,3,0,0,0,771332.8,5838228.4
3,2015-01-01 01:00:22,46.382,6.242,0,0,0,0,0,0,0,0,0,694856.3,5841777.7
4,2015-01-01 01:00:34,46.160,8.777,0,0,0,0,0,0,0,0,0,977051.2,5806026.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115888,2018-09-06 16:51:13,46.799,8.232,0,0,0,0,0,0,0,0,0,916382.0,5909327.2
1115889,2018-09-06 16:53:31,46.200,6.146,0,0,0,0,0,0,0,0,0,684169.6,5812457.5
1115890,2018-09-06 16:53:53,46.231,6.103,0,0,0,0,0,0,0,0,0,679382.9,5817444.7
1115891,2018-09-06 16:54:32,47.050,7.283,0,0,0,0,0,0,0,0,0,810739.9,5950239.2


In [232]:
def bkapp(doc):
    
    #Create the Timeline figure
    Timeline = figure(background_fill_color="#fafafa",
                      x_axis_type='datetime',
                      y_axis_type=None,
                      tools="",toolbar_location=None,
                      plot_width=int(screen_length*0.8),
                      plot_height=int(screen_height*0.2))
    
    range_tool = RangeTool(x_range=Range1d(start=TMH_data.iloc[0]['created_at_CET'],end=TMH_data.iloc[0-1]['created_at_CET']))
    range_tool.overlay.fill_color = "navy"
    range_tool.overlay.fill_alpha = 0.2
    
    Timeline.add_tools(range_tool)
    Timeline.toolbar.active_multi = range_tool
    
    Timeline.circle()

    #load the background of the map plot
    tile_provider = get_provider(CARTODBPOSITRON)

    #Create the Map figure
    # range bounds supplied in web mercator coordinates
    Map = figure(x_range=(650000, 1200000), y_range=(5700000, 6100000),
                 x_axis_type="mercator", y_axis_type="mercator",
                 plot_width=int(screen_length*0.4),
                 plot_height=int(screen_height*0.6))
    
    #Add the background to the map
    Map.add_tile(tile_provider)
    
    source = ColumnDataSource(data=dict(longitude=TMH_data['x_mercator'][0:1000], latitude=TMH_data['y_mercator'][0:1000]))

    Map.circle(x='longitude', y='latitude', size=2, color="black", alpha=0.7, source=source)
    
    #Create the Histogram figure
    Histogram = figure(background_fill_color="#fafafa",
                       plot_width=int(screen_length*0.4),
                       plot_height=int(screen_height*0.6))
    
    Histogram.circle(1,1)
    

    plots = layout([
        [Timeline],
        [Map,Histogram],
        
    ])
    doc.add_root(plots)

In [233]:
show(bkapp)

In [216]:
TMH_data['created_at_CET'][0]

Timestamp('2015-01-01 01:00:03')

In [207]:
%%time
TMH_data.sort_values(by='created_at_CET')

Wall time: 1.33 s


,created_at_CET,latitude,longitude,Overall.score,Anger,Confusion,Disgust,Fear,Happiness,Sadness,Shame,Surprise,x_mercator,y_mercator
0,2015-01-01 01:00:03,47.458,8.548,0,0,0,0,0,0,0,0,0,951559.0,6017154.4
1,2015-01-01 01:00:09,46.160,8.777,0,0,0,0,0,0,0,0,0,977051.2,5806026.5
2,2015-01-01 01:00:14,46.360,6.929,3,0,0,0,0,3,0,0,0,771332.8,5838228.4
3,2015-01-01 01:00:22,46.382,6.242,0,0,0,0,0,0,0,0,0,694856.3,5841777.7
4,2015-01-01 01:00:34,46.160,8.777,0,0,0,0,0,0,0,0,0,977051.2,5806026.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115888,2018-09-06 16:51:13,46.799,8.232,0,0,0,0,0,0,0,0,0,916382.0,5909327.2
1115889,2018-09-06 16:53:31,46.200,6.146,0,0,0,0,0,0,0,0,0,684169.6,5812457.5
1115890,2018-09-06 16:53:53,46.231,6.103,0,0,0,0,0,0,0,0,0,679382.9,5817444.7
1115891,2018-09-06 16:54:32,47.050,7.283,0,0,0,0,0,0,0,0,0,810739.9,5950239.2


In [213]:
%%time
TMH_data.sort_values(by='created_at_CET')

Wall time: 174 ms


,created_at_CET,latitude,longitude,Overall.score,Anger,Confusion,Disgust,Fear,Happiness,Sadness,Shame,Surprise,x_mercator,y_mercator
0,2015-01-01 01:00:03,47.458,8.548,0,0,0,0,0,0,0,0,0,951559.0,6017154.4
1,2015-01-01 01:00:09,46.160,8.777,0,0,0,0,0,0,0,0,0,977051.2,5806026.5
2,2015-01-01 01:00:14,46.360,6.929,3,0,0,0,0,3,0,0,0,771332.8,5838228.4
3,2015-01-01 01:00:22,46.382,6.242,0,0,0,0,0,0,0,0,0,694856.3,5841777.7
4,2015-01-01 01:00:34,46.160,8.777,0,0,0,0,0,0,0,0,0,977051.2,5806026.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115888,2018-09-06 16:51:13,46.799,8.232,0,0,0,0,0,0,0,0,0,916382.0,5909327.2
1115889,2018-09-06 16:53:31,46.200,6.146,0,0,0,0,0,0,0,0,0,684169.6,5812457.5
1115890,2018-09-06 16:53:53,46.231,6.103,0,0,0,0,0,0,0,0,0,679382.9,5817444.7
1115891,2018-09-06 16:54:32,47.050,7.283,0,0,0,0,0,0,0,0,0,810739.9,5950239.2


In [210]:
TMH_data['created_at_CET'] = pd.to_datetime(TMH_data['created_at_CET'], format='%Y-%m-%d %H:%M:%S')

In [231]:
Range1d(start=TMH_data.iloc[0]['created_at_CET'],end=TMH_data.iloc[0-1]['created_at_CET'])

Range1d(id='10338', ...)

In [230]:
TMH_data.iloc[0]['created_at_CET']

Timestamp('2015-01-01 01:00:03')

In [179]:
import numpy as np

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool, CustomJS, CDSView, CustomJSFilter
from bokeh.plotting import figure, show
from bokeh.sampledata.stocks import AAPL

dates = np.array(AAPL['date'], dtype=np.datetime64)
source = ColumnDataSource(data=dict(date=dates, close=AAPL['adj_close'], volume=AAPL['volume']))

p = figure(plot_height=300, x_axis_type="datetime", x_range=(dates[1500], dates[2500]))

p.line('date', 'close', source=source)

range_tool = RangeTool(x_range=p.x_range)

p.x_range.callback = CustomJS(args=dict(source=source), code="source.change.emit();")

date_filter = CustomJSFilter(args=dict(source=source, x_range=p.x_range), code="""
    let start=x_range.start;
    let end=x_range.end;
    let dates = source.data['date'];
    let indices = [];
    for (var i = 0; i <= dates.length; i++){
        if (dates[i] >= start && dates[i] <= end) indices.push(i);
    }
    return indices;
    """)

view = CDSView(source=source, filters=[date_filter])

p1 = figure(plot_height=300)
p1.circle (x='close', y='volume', source=source, view=view)

select = figure(plot_height=130, x_axis_type="datetime")

select.line('date', 'close', source=source)
select.add_tools(range_tool)

show(column(p, p1, select))

AttributeError: unexpected attribute 'callback' to Range1d, similar attributes are js_event_callbacks